## Basic Setting

In [4]:
### 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Font directories:
	/root/.local/share/fonts
	/usr/local/share/fonts
	/usr/share/fonts
	/root/.fonts
	/usr/share/fonts/truetype
	/usr/share/fonts/truetype/dejavu
	/usr/share/fonts/truetype/liberation
	/usr/share/fonts/truetype/nanum
/root/.local/share/fonts: skipping, no such directory
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/dejavu: caching, new cache contents: 22 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/root/.fonts: skipping, no such directory
/usr/shar

In [5]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

In [6]:
import pandas as pd
import seaborn as sns

# Persona2 DDA

## 희귀질환 Data(수치형)

### Dataset
- 2021년 data
  + 희귀질환 질병분류코드별 "항목별 진료내역", "지역별 사망자수", "연령별 사망자수", "지역별 발생자수", "성별별 + 연령별 발생자수"
  + 희귀질환 건강보험 진료비 통계
- 2022년 data
  + 중증질환 "연령별 + 성별별 + 진료현황", 4대 중증질환 약품비 현황
  + 희귀질환 건강보험 진료비 통계

### 발생자 및 사망자 수 data 

#### 질병분류코드별 발생자수 data

In [57]:
df_gender_age_accrual = pd.read_csv('../../csv/2021_전국_질병분류코드별_성별_연령군별_발생자_수_발생자_수_200명_초과_질환__20240424095607.csv', header=1)
df_gender_age_accrual[:5]

,질병코드 및 질환별(1),성별(1),발생자수,<1,1~9,10~19,20~29,30~39,40~49,50~59,60~69,70~79,80<=,알수없음
0,계,계,43079,466,799,2272,4196,4344,5345,7072,8618,6602,3365,0
1,D61.3 특발성 무형성빈혈,계,406,1,14,30,46,36,51,65,74,65,24,0
2,D61.3 특발성 무형성빈혈,남자,186,0,8,17,22,12,26,26,37,30,8,0
3,D61.3 특발성 무형성빈혈,여자,220,1,6,13,24,24,25,39,37,35,16,0
4,D61.9 저형성빈혈 NOS,계,223,0,13,18,18,29,27,31,42,26,19,0


|value|contents|key|opiniㅋon|
|--|--|--|--|
|질병분류코드 및 질환별(1)|질환코드/질환명 분류 ex)A81.0 크로이츠펠트-야콥병|-|범주형, 코드와 질환명 분류 필요|
|성별(1)|성별 및 총계|남자, 여자, 계|남자, 여자와 계를 분리해야 함.|
|발생자수|발생자수|-|수치형|
|<1|1세 미만|-|수치형|
|1~9|1에서 9세|-|수치형|
|10~19|10에서 19세|-|수치형|
|20~29|20에서 29세|-|수치형|
|30~39|30에서 39세|-|수치형|
|40~49|40에서 49세|-|수치형|
|50~59|50에서 59세|-|수치형|
|60~69|60에서 69세|-|수치형|
|70~79|70에서 79세|-|수치형|
|80<=|80 이상|-|수치형|
|알수없음|알수없음|-|수치형이나 0밖에 없음 - 삭제 필요|

##### 기본 전처리

In [58]:
def split_codes(x):
    # 문자열을 공백을 기준으로 두 부분으로 나누기
    parts = x.split(maxsplit=1)
    # 두 부분으로 나눌 수 있으면 해당 값을 반환, 그렇지 않으면 code만 반환하고 name은 빈 문자열로 처리
    if x in '질병의':
        parts = (none,none)
        return pd.Series(parts)
        
    else:
        return pd.Series(parts)  # name을 빈 문자열로 처리

In [59]:
df_gender_age_accrual[['질병코드', '질병명']] = df_gender_age_accrual['질병코드 및 질환별(1)'].apply(split_codes)

In [60]:
df_gender_age_accrual

,질병코드 및 질환별(1),성별(1),발생자수,<1,1~9,10~19,20~29,30~39,40~49,50~59,60~69,70~79,80<=,알수없음,질병코드,질병명
0,계,계,43079,466,799,2272,4196,4344,5345,7072,8618,6602,3365,0,계,NaN
1,D61.3 특발성 무형성빈혈,계,406,1,14,30,46,36,51,65,74,65,24,0,D61.3,특발성 무형성빈혈
2,D61.3 특발성 무형성빈혈,남자,186,0,8,17,22,12,26,26,37,30,8,0,D61.3,특발성 무형성빈혈
3,D61.3 특발성 무형성빈혈,여자,220,1,6,13,24,24,25,39,37,35,16,0,D61.3,특발성 무형성빈혈
4,D61.9 저형성빈혈 NOS,계,223,0,13,18,18,29,27,31,42,26,19,0,D61.9,저형성빈혈 NOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,"Q85.0 신경섬유종증(비악성) 1형, 2형",남자,146,13,57,21,17,15,7,11,4,1,0,0,Q85.0,"신경섬유종증(비악성) 1형, 2형"
153,"Q85.0 신경섬유종증(비악성) 1형, 2형",여자,174,20,55,14,28,21,10,17,8,1,0,0,Q85.0,"신경섬유종증(비악성) 1형, 2형"
154,"질병의 특성상 다양한 장기에, 다양한 병변이 발생되는 질환으로, 진단을 통해 확인된...",계,426,0,0,2,6,15,40,107,150,84,22,0,질병의,"특성상 다양한 장기에, 다양한 병변이 발생되는 질환으로, 진단을 통해 확인된 병태에..."
155,"질병의 특성상 다양한 장기에, 다양한 병변이 발생되는 질환으로, 진단을 통해 확인된...",남자,302,0,0,1,4,6,31,69,106,67,18,0,질병의,"특성상 다양한 장기에, 다양한 병변이 발생되는 질환으로, 진단을 통해 확인된 병태에..."


#### 지역별 질병분류코드별 발생자수 data

In [11]:
df_region_accrual = pd.read_csv('../../csv/2021_지역별_질병분류코드별_발생자_수_발생자_수_200명_초과_질환__20240424095500.csv')
df_region_accrual[:5]

,질병코드 및 질환별(1),2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11,2021.12,2021.13,2021.14,2021.15,2021.16,2021.17,2021.18
0,질병코드 및 질환별(1),발생자 수,서울특별시,인천광역시,경기도,강원도,충청남도,충청북도,대전광역시,세종특별자치시,경상남도,경상북도,부산광역시,대구광역시,울산광역시,전라북도,전라남도,광주광역시,제주특별자치도,알수없음
1,계,43079,8601,2446,11377,1274,1752,1291,1080,238,2431,2243,2762,2120,789,1501,1392,1076,690,16
2,D61.3 특발성 무형성빈혈,406,73,29,101,20,17,9,12,3,24,19,15,22,6,6,20,20,9,1
3,D61.9 저형성빈혈 NOS,223,47,34,56,4,9,2,1,2,21,7,15,11,4,6,2,1,1,0
4,D68.6 항인지질증후군,469,126,27,121,12,20,3,10,1,11,30,13,37,4,13,13,14,14,0


|value|contents|key|opinion|
|--|--|--|--|
|질병분류코드 및 질환별(2)|질환코드/질환명 분류 ex)A81.0 크로이츠펠트-야콥병, 계|-|범주형, 코드와 질환명 분류 필요, 계 정보 따로 빼야 함|
|발생자 수|발생자 수|-|수치형|
|서울특별시|서울특별시|-|수치형|
|인천광역시|인천광역시|-|수치형|
|경기도|경기도|-|수치형|
|강원도|강원도|-|수치형|
|충청남도|충청남도|-|수치형|
|충청북도|충청북도|-|수치형|
|대전광역시|대전광역시|-|수치형|
|세종특별자치시|세종특별자치시|-|수치형|
|부산광역시|부산광역시|-|수치형|
|대구광역시|대구광역시|-|수치형|
|울산광역시|울산광역시|-|수치형|
|전라북도|전라북도|-|수치형|
|전라남도|전라남도|-|수치형|
|광주광역시|광주광역시|-|수치형|
|제주특별자치도|제주특별자치도|-|수치형|
|알수없음|알수없음|-|수치형|

#### 전국 질병분류코드별 연령군별 사망자수 data

In [12]:
df_age_death = pd.read_csv('../../csv/2021_전국_질병분류코드별_연령군별_사망자_수_20240424100309.csv')
df_age_death[:5]

,질병분류코드 및 질환별(1),질병분류코드 및 질환별(2),2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6
0,질병분류코드 및 질환별(1),질병분류코드 및 질환별(2),사망자수,<1,1-14,15-24,25-44,45-64,65≤
1,계,소계,1845,34,11,15,65,344,1376
2,희귀질환,소계,1826,30,11,14,65,337,1369
3,희귀질환,A81.0 크로이츠펠트-야콥병,29,0,0,0,0,8,21
4,희귀질환,D59.3 비정형 용혈-요독증후군,4,0,0,0,0,0,4


|value|contents|key|opinion|
|--|--|--|--|
|질병분류코드 및 질환별(1)|희귀질환 종류|희귀질환, 극희귀질환, 기타염색체이상질환|범주형|
|질병분류코드 및 질환별(2)|질환코드/질환명 분류 ex)A81.0 크로이츠펠트-야콥병|-|범주형, 코드와 질환명 분류 필요|
|사망자수|사망자수|-|수치형|
|<1|1세 미만|-|수치형|
|1-14|1에서 14세|-|수치형|
|15-24|15에서 24세|-|수치형|
|25-44|25에서 44세|-|수치형|
|45-64|45에서 64세|-|수치형|
|65≤|65세 이상|-|수치형|

#### 지역별 질병분류코드별 사망자수 data

In [13]:
df_region_death = pd.read_csv('../../csv/2021_지역별_질병분류코드별_사망자_수_20240424100217.csv')
df_region_death[:5]

,질병분류코드 및 질환별(1),질병분류코드 및 질환별(2),2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11,2021.12,2021.13,2021.14,2021.15,2021.16,2021.17
0,질병분류코드 및 질환별(1),질병분류코드 및 질환별(2),사망자 수,서울특별시,인천광역시,경기도,강원도,충청남도,충청북도,대전광역시,세종특별자치시,경상남도,경상북도,부산광역시,대구광역시,울산광역시,전라북도,전라남도,광주광역시,제주특별자치도
1,계,소계,1845,339,106,452,77,79,76,36,14,105,88,134,85,31,84,76,39,24
2,희귀질환,소계,1826,332,104,450,76,77,75,36,14,104,87,133,85,31,84,76,38,24
3,희귀질환,A81.0 크로이츠펠트-야콥병,29,6,2,11,0,0,0,1,0,1,0,3,0,0,0,3,2,0
4,희귀질환,D59.3 비정형 용혈-요독증후군,4,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0


|value|contents|key|opinion|
|--|--|--|--|
|질병분류코드 및 질환별(1)|희귀질환 종류|희귀질환, 극희귀질환, 기타염색체이상질환, 계|범주형, 계 정보 따로 빼야 함.|
|질병분류코드 및 질환별(2)|질환코드/질환명 분류 ex)A81.0 크로이츠펠트-야콥병, 소계|-|범주형, 코드와 질환명 분류 필요, 소계 정보 따로 빼야 함|
|사망자수|사망자수|-|수치형|
|서울특별시|서울특별시|-|수치형|
|인천광역시|인천광역시|-|수치형|
|경기도|경기도|-|수치형|
|강원도|강원도|-|수치형|
|충청남도|충청남도|-|수치형|
|충청북도|충청북도|-|수치형|
|대전광역시|대전광역시|-|수치형|
|세종특별자치시|세종특별자치시|-|수치형|
|부산광역시|부산광역시|-|수치형|
|대구광역시|대구광역시|-|수치형|
|울산광역시|울산광역시|-|수치형|
|전라북도|전라북도|-|수치형|
|전라남도|전라남도|-|수치형|
|광주광역시|광주광역시|-|수치형|
|제주특별자치도|제주특별자치도|-|수치형|

### 진료비용, 보험청구비용 data 

#### 희귀질환 data

##### 질병분류코드 + 요양급여비용 항목별 진료내역 data

In [16]:
df_medical_expense = pd.read_csv('../../csv/2021_질병분류코드별_진료내역_항목별_요양급여비용_현황_등록_후_12개월__20240424095654.csv')
df_medical_expense[:5]

,질병분류코드 및 질환별(1),질병분류코드 및 질환별(2),2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11
0,질병분류코드 및 질환별(1),질병분류코드 및 질환별(2),계,진찰료,입원료,투약료,주사료,마취료,이학요법료,정신요법료,처치 및 수술료,검사료,영상 및 방사선 치료료,기타
1,계,소계,317789679,63604106,48547544,6489961,69347268,1024116,13417865,129127,22622419,46850108,4686879,41070288
2,희귀질환,소계,308572698,62306518,46864613,6321592,67957685,982444,12554025,116157,21761403,45152854,4559229,39996179
3,희귀질환,A81.0 크로이츠펠트-야콥병,1624857,73975,273196,29930,124240,1114,107553,20,134960,156167,7086,716615
4,희귀질환,D12.6 M8220/0 가족성선종성폴립증,322142,25918,51745,3271,16059,10331,70,240,114390,52848,3312,43957


|value|contents|key|opinion|
|--|--|--|--|
|질병분류코드 및 질환별(1)|희귀질환 종류|희귀질환, 극희귀질환, 기타염색체이상질환|범주형|
|질병분류코드 및 질환별(2)|질환코드/질환명 분류 ex)A81.0 크로이츠펠트-야콥병|-|범주형, 코드와 질환명 분류 필요|
|계|아래 항목들의 총액|-|수치형|
|진찰료|진찰료|-|수치형|
|입원료|입원료|-|수치형|
|투약료|투약료|-|수치형|
|주사료|주사료|-|수치형|
|마취료|마취료|-|수치형|
|이학요법료|물리치료료-비급여|-|수치형|
|정신요법료|정신치료- 비급여|-|수치형|
|처치 및 수술료|처치 및 수술료 - 비급여|-|수치형|
|검사료|검사료 - 비급여|-|수치형|
|영상 및 방사선 치료료|영상 및 방사선 치료료 - 비급여|-|수치형|
|기타|기타|-|수치형|

##### 질병분류코드별 건강보험 청구 및 요양급여비용 내역 data

In [18]:
df_health_insurance_2021 = pd.read_csv('../../csv/건강보험심사평가원_희귀질환 진료비 통계 2021.csv', encoding='CP949')
df_health_insurance_2021[:5]

,진료년도,희귀질환 상병코드,환자수,명세서 청구건수,입내원일수,보험자부담금,요양급여비용총액
0,2021,D66,1671,31653,33590,134364000000,148528000000
1,2021,D67,389,6066,6461,50277576220,55536599090
2,2021,D70,14960,32471,56526,14622808620,16707794010
3,2021,D71,119,910,1839,638847540,690840100
4,2021,G10,324,1800,12292,991135350,1193086390


|value|contents|key|opinion|
|--|--|--|--|
|진료년도|연도 정보|2021|2021년도 정보밖에 없으니 삭제|
|희귀질환 상병코드|질환별 코드|A00 - Q99|opinion|
|환자수|환자수|-|수치형|
|명세서 청구건수|명세서 청구건수|-|수치형|
|입내원일수|입내원일수|-|수치형|
|보험자부담금|건강보험료|-|수치형|
|요양급여비용총액|요양급여비용총액|-|수치형|

In [19]:
df_health_insurance_2022 = pd.read_csv('../../csv/건강보험심사평가원_희귀질환 건강보험 진료 통계_20221231.csv', encoding='CP949')
df_health_insurance_2022[:5]

,진료년도,희귀질환 상병코드,환자수,명세서 청구건수,입내원일수,보험자부담금,요양급여비용총액
0,2022,D66,1718,30492,31616,140000000000,155000000000
1,2022,D67,395,5977,6200,54740079160,60505539180
2,2022,D70,16914,36070,56966,13696287940,15919579930
3,2022,D71,107,973,1786,528293110,591926350
4,2022,G10,347,1874,13123,1111407030,1335467120


|value|contents|key|opinion|
|--|--|--|--|
|진료년도|연도 정보|2021|2021년도 정보밖에 없으니 삭제|
|희귀질환 상병코드|질환별 코드|A00 - Q99|opinion|
|환자수|환자수|-|수치형|
|명세서 청구건수|명세서 청구건수|-|수치형|
|입내원일수|입내원일수|-|수치형|
|보험자부담금|건강보험료|-|수치형|
|요양급여비용총액|요양급여비용총액|-|수치형|

#### 중증질환 data

##### 4대 중증질환(암, 뇌혈관, 심장, 휘귀/중증난치질환) 약품비 data

In [20]:
df_serious_disease_medicine = pd.read_csv('../../csv/2022_4대_중증질환_약품비_건강보험__20240424094151.csv')
df_serious_disease_medicine[:5]

,중증질환 구분(1),2022,2022.1,2022.2,2022.3,2022.4,2022.5
0,중증질환 구분(1),계,계,입원,입원,외래,외래
1,중증질환 구분(1),청구건수 (천건),청구금액 (백만원),청구건수 (천건),청구금액 (백만원),청구건수 (천건),청구금액 (백만원)
2,계,25069.0,5849552,2376.0,1440835,22692.0,4408717
3,암질환,9827.0,3117039,1595.0,1068558,8232.0,2048481
4,뇌혈관질환,78.0,65588,77.0,65511,0.6,77


In [21]:
df_serious_disease_medicine['중증질환 구분(1)'].unique()

array(['중증질환 구분(1)', '계', '암질환', '뇌혈관질환', '심장질환', '희귀·중증난치질환'],
      dtype=object)

|value|contents|key|opinion|
|--|--|--|--|
|중증질환 구분(1)|중증질환 구분|'계', '암질환', '뇌혈관질환', '심장질환', '희귀·중증난치질환'|계 제외하기|
|계 - 청구건수(천건)|전체 청구건수|A00 - Q99|opinion|
|계 - 청구금액(백만원)|전체 청구금액|-|수치형|
|입원 - 청구건수(천건)	|입원 청구건수|-|수치형|
|입원 - 청구금액(백만원)	|입원 청구금액|-|수치형|
|외래 - 청구건수(천건)|외래 청구건수|-|수치형|
|외래 - 청구금액(백만원)|외래 청구금액|-|수치형|

##### 중증질환(암, 뇌혈관, 심장, 휘귀/중증난치질환, 화상, 외상, 결핵) 진료비용 data

In [31]:
df_gender_age_serious_disease_medical_expense = pd.read_csv('../../csv/2022_중증질환_산정특례적용_연령별_성별_진료현황_20240424093755.csv')
df_gender_age_serious_disease_medical_expense.head()

,연령별(1),성별(1),2022,2022.1,2022.2,2022.3,2022.4,2022.5,2022.6,2022.7,...,2022.40,2022.41,2022.42,2022.43,2022.44,2022.45,2022.46,2022.47,2022.48,2022.49
0,연령별(1),성별(1),계,계,계,계,계,암질환,암질환,암질환,...,결핵,결핵,결핵,결핵,결핵,잠복결핵,잠복결핵,잠복결핵,잠복결핵,잠복결핵
1,연령별(1),성별(1),진료실인원수 (명),입내원일수 (일),요양급여일수 (일),진료비 (천원),급여비 (천원),진료실인원수 (명),입내원일수 (일),요양급여일수 (일),...,진료실인원수 (명),입내원일수 (일),요양급여일수 (일),진료비 (천원),급여비 (천원),진료실인원수 (명),입내원일수 (일),요양급여일수 (일),진료비 (천원),급여비 (천원)
2,계,계,2569669,67044120,617580504,20955584856,19026420452,1370510,33663687,288731422,...,28567,577508,4046379,141808388,137867480,33639,223528,2577582,8867768,8826062
3,계,남자,1213016,32266545,278510470,11274973918,10269419580,633417,15917584,127206540,...,17230,342010,2443988,88233760,85842208,11988,78358,936375,3214308,3202047
4,계,여자,1356653,34777575,339070034,9680610939,8757000874,737093,17746103,161524882,...,11337,235498,1602391,53574627,52025272,21651,145170,1641207,5653460,5624015


전체적으로 전처리 필수
|value|contents|key|opinion|
|--|--|--|--|
|연령별(1)|중증질환 구분|-|계 제외하기|
|성별(1)|전체 청구건수|-|opinion|
|계 - 진료실인원수 (명)|총 진료실인원수(명)|-|수치형|
|계 - 입내원일수 (일)|총 입내원일수 (일)|-|수치형|
|계 -	요양급여일수 (일)|총 요양급여일수 (일)|-|수치형|
|계 - 진료비 (천원)	|총 진료비 (천원)|-|수치형|
|계 - 급여비 (천원)|총 급여비 (천원)|-|수치형|
|암질환 - 진료실인원수 (명)|암질환의 진료실인원수 (명)|-|수치형|
|암질환 - 입내원일수 (일)|암질환의 입내원일수 (일|-|수치형|
|암질환 - 요양급여일수 (일)|암질환의 요양급여일수 (일)|-|수치형|
|암질환 -  진료비 (천원)|암질환의 진료비 (천원)|-|수치형|
|암질환 -  급여비 (천원)|암질환의 급여비 (천원)|-|수치형|
|뇌혈관 - 진료실인원수 (명)|뇌혈관의 진료실인원수 (명)|-|수치형|
|뇌혈관 - 입내원일수 (일)|뇌혈관의 입내원일수 (일|-|수치형|
|뇌혈관 - 요양급여일수 (일)|뇌혈관의 요양급여일수 (일)|-|수치형|
|뇌혈관 -  진료비 (천원)|뇌혈관의 진료비 (천원)|-|수치형|
|뇌혈관 -  급여비 (천원)|뇌혈관의 급여비 (천원)|-|수치형|
|희귀난치성질환 - 진료실인원수 (명)|희귀난치성질환의 진료실인원수 (명)|-|수치형|
|희귀난치성질환 - 입내원일수 (일)|희귀난치성질환의 입내원일수 (일|-|수치형|
|희귀난치성질환 - 요양급여일수 (일)|희귀난치성질환의 요양급여일수 (일)|-|수치형|
|희귀난치성질환 -  진료비 (천원)|희귀난치성질환의 진료비 (천원)|-|수치형|
|희귀난치성질환 -  급여비 (천원)|희귀난치성질환의 급여비 (천원)|-|수치형|
|중증화상 - 진료실인원수 (명)|중증화상의 진료실인원수 (명)|-|수치형|
|중증화상 - 입내원일수 (일)|중증화상의 입내원일수 (일|-|수치형|
|중증화상 - 요양급여일수 (일)|중증화상의 요양급여일수 (일)|-|수치형|
|중증화상 -  진료비 (천원)|중증화상의 진료비 (천원)|-|수치형|
|중증화상 -  급여비 (천원)|중증화상의 급여비 (천원)|-|수치형|
|결핵 - 진료실인원수 (명)|결핵의 진료실인원수 (명)|-|수치형|
|결핵 - 입내원일수 (일)|결핵의 입내원일수 (일|-|수치형|
|결핵 - 요양급여일수 (일)|결핵의 요양급여일수 (일)|-|수치형|
|결핵 -  진료비 (천원)|결핵의 진료비 (천원)|-|수치형|
|결핵 -  급여비 (천원)|결핵의 급여비 (천원)|-|수치형|
|잠복결핵 - 진료실인원수 (명)|잠복결핵의 진료실인원수 (명)|-|수치형|
|잠복결핵 - 입내원일수 (일)|잠복결핵의 입내원일수 (일|-|수치형|
|잠복결핵 - 요양급여일수 (일)|잠복결핵의 요양급여일수 (일)|-|수치형|
|잠복결핵 -  진료비 (천원)|잠복결핵의 진료비 (천원)|-|수치형|
|잠복결핵 -  급여비 (천원)|잠복결핵의 급여비 (천원)|-|수치형|
